In [1]:
##############################################################################
#####################################################################
#####################################################################
#
# DATA CLEANING: 
#                 null-values and object dtypes
#
#####################################################################
##############################################################################

In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp

 # importing raw input data
raw = pd.read_csv("../data/train.csv")
    # confirm
raw.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [9]:
# safety-first:

    # copy df into w(orking)df:
wdf = raw.copy()
    # make sure vals in col"id" are all unique:
len(wdf['Id'].unique())==len(wdf)

True

In [10]:
# add date col:

wdf = wdf.assign(date=wdf['MoSold'].astype(str)+"-"+wdf['YrSold'].astype(str))
    # translate to datetime + update df 
wdf['date'] = pd.to_datetime(wdf.date)
        # rename important cols with short and easy names + sort by date, price
wdf = wdf.rename(columns={'Id':'id','SalePrice':'price', 'YrSold':'year', 'MoSold':'month'}).sort_values(['date','price']) 

In [11]:
# add constant:

wdf = sm.add_constant(wdf)

# NA Values

In [12]:
# null values:

    # storing 50% len of raw df 
halfdf = len(raw)/2

    # new df for count of na-vals
nasumdf = pd.DataFrame(wdf.isna().sum())
nasumdf = nasumdf.reset_index().rename(columns={'index': 'col',0:'sum_of_na'}).sort_values('col')
nasumdf = nasumdf.loc[nasumdf['sum_of_na']>0].reset_index()
    
    # new df for all cols with > 50% na
naover50 = nasumdf.loc[nasumdf['sum_of_na']>halfdf]
    # review:
naover50

,index,col,sum_of_na
0,7,Alley,1369
7,74,Fence,1179
17,75,MiscFeature,1406
18,73,PoolQC,1453


In [13]:
    # new df to store col with > 25% na vals
naover25 = nasumdf.loc[nasumdf['sum_of_na']>halfdf/2]
    #review
naover25

,index,col,sum_of_na
0,7,Alley,1369
7,74,Fence,1179
8,58,FireplaceQu,690
17,75,MiscFeature,1406
18,73,PoolQC,1453


In [14]:
##############################################################################
###  NA HANDELING:
#
# ⟹ observation: 4 cols contain over 50% na vals, 1 over 25%
#    ⟹ conclusion: info value not not sufficient for further analysis 
#    ⟹ decision: drop cols Alley, Fence, MiscFeature, PoolQC
# ⟹ observation: 14 more cols need further investigation
#    ⟹ next step: cols saved in 'investigate' value inspection
#
##############################################################################

    # saving cols to drop in list
colstodrop = naover25['col'].to_list()

    # drop and update wdf
wdf = wdf.drop(columns=[c for c in colstodrop])

In [15]:
nasumdf2 = pd.DataFrame(wdf.isna().sum()).reset_index().rename(columns={'index':'col',0:'nas'}).sort_values('nas',ascending=False)
nasumdf2 = nasumdf2.loc[nasumdf2['nas']>0]
    # review
nasumdf2
        # observation: 14 cols still have na vals
            # decision: drop rows containing na, if these rows make <= 5.5% of len(raw)

,col,nas
4,LotFrontage,259
58,GarageYrBlt,81
63,GarageCond,81
57,GarageType,81
59,GarageFinish,81
62,GarageQual,81
35,BsmtFinType2,38
32,BsmtExposure,38
33,BsmtFinType1,37
31,BsmtCond,37


In [16]:
    # storing %5.5 of len(raw) in fivepc
fivepc = (len(raw)/100)*5.5

In [17]:
    # save 5.5% na in df
fivepcdf = nasumdf2.loc[nasumdf2['nas']<fivepc]
    # storing cols with < 5% na vals in list:
colfivepc = fivepcdf['col'].to_list()
    # review: 
# out=
colfivepc = ['BsmtFinType2', 'BsmtExposure', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrArea', 'MasVnrType', 'Electrical', 'GarageYrBlt','GarageFinish', 'GarageQual', 'GarageCond', 'GarageType']


In [18]:
# drop fivepc rows and update wdf:
for col in colfivepc:
    l = wdf.loc[wdf[col].isna()].index.to_list()
    wdf = wdf.drop(l)

In [19]:
# review remaining na sums
nasumdf3 = wdf.isna().sum().reset_index().rename(columns={'index':'col',0:'nas'}).sort_values('nas',ascending=False)
nasumdf3 = nasumdf3.loc[nasumdf3['nas']>0]
nasumdf3['col'].to_list()

    # observation: only one col remains : LotFrontage

['LotFrontage']

In [20]:
    # closer look at col LotFrontage
wdf['LotFrontage'].value_counts()

60.0     120
80.0      67
70.0      63
75.0      50
50.0      49
        ... 
150.0      1
144.0      1
160.0      1
46.0       1
141.0      1
Name: LotFrontage, Length: 107, dtype: int64

In [21]:
wdf['LotFrontage'].min()

21.0

In [22]:
wdf['LotFrontage'].max()

313.0

In [23]:
wdf['LotFrontage'].mean()

70.75959780621572

In [24]:
wdf['LotFrontage'].median()

70.0

In [25]:
wdf['LotFrontage'].std()

24.508858570320587

In [26]:
    # review concerned rows
wdf.loc[wdf['LotFrontage'].isna()]

,const,id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,month,year,SaleType,SaleCondition,price,date
996,1.0,997,20,RL,NaN,10659,Pave,IR1,Lvl,AllPub,...,0,0,0,0,1,2006,COD,Normal,136500,2006-01-01
370,1.0,371,60,RL,NaN,8121,Pave,IR1,Lvl,AllPub,...,0,0,0,0,1,2006,WD,Normal,172400,2006-01-01
169,1.0,170,20,RL,NaN,16669,Pave,IR1,Lvl,AllPub,...,0,0,0,0,1,2006,WD,Normal,228000,2006-01-01
545,1.0,546,50,RL,NaN,13837,Pave,IR1,Lvl,AllPub,...,0,0,0,0,2,2006,WD,Normal,229000,2006-02-01
1033,1.0,1034,20,RL,NaN,8125,Pave,Reg,Lvl,AllPub,...,0,0,0,0,2,2006,WD,Normal,230000,2006-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,1.0,1359,160,FV,NaN,2117,Pave,Reg,Lvl,AllPub,...,0,0,0,0,6,2010,WD,Normal,177500,2010-06-01
470,1.0,471,120,RL,NaN,6820,Pave,IR1,Lvl,AllPub,...,0,140,0,0,6,2010,WD,Normal,212000,2010-06-01
412,1.0,413,20,FV,NaN,4403,Pave,IR2,Lvl,AllPub,...,0,0,0,0,6,2010,New,Partial,222000,2010-06-01
939,1.0,940,70,RL,NaN,24090,Pave,Reg,Lvl,AllPub,...,0,0,0,0,6,2010,COD,Normal,244400,2010-06-01


In [27]:
###################################################################################################################
### NOTES:
#
# observation: col 'LotArea' has values where 'LotFrontage' is na
    # decision: try to figure out if there is some correlation between these two
###################################################################################################################

    # get these cols and save them to lotdf:
    
lotdf = wdf.copy()
lotdf = lotdf.get(['LotFrontage','LotArea']).dropna().sort_values('LotArea')

In [28]:
###################################################################################################################
### NOTES:
# 
# consideration:
#        if all lots would be squares, the lenght the frontages would be square-roots of each areas
# 
# conclusion:
#        assign a col with frontage-lenghts if areas were squares to review deviations
###################################################################################################################

In [29]:
    # assign col with lenght if area is suare
lotdf = lotdf.assign(ifsqare= np.sqrt(lotdf['LotArea']))

    # assign col with difference beween real frontage and squarefrontage
lotdf = lotdf.assign(diff=lotdf['ifsqare'] - lotdf['LotFrontage']).sort_values('diff')

# review key values of difference:

In [30]:
lotdf['diff'].std()

22.437757978655178

In [31]:
lotdf['diff'].max()

313.9450398484717

In [32]:
lotdf['diff'].min()

-146.7171085168411

In [33]:
lotdf['diff'].mean()

26.634305902799507

In [34]:
lotdf['diff'].median()

24.810547537889594

In [35]:
# closer look at highest diff
lotdf.tail(10)

,LotFrontage,LotArea,ifsqare,diff
1409,46.0,20544,143.331783,97.331783
692,42.0,26178,161.796168,119.796168
271,73.0,39104,197.747314,124.747314
1184,50.0,35133,187.437990,137.437990
848,75.0,45600,213.541565,138.541565
53,68.0,50271,224.211953,156.211953
661,52.0,46589,215.844852,163.844852
769,47.0,53504,231.309317,184.309317
451,62.0,70761,266.009398,204.009398
313,150.0,215245,463.945040,313.945040


In [36]:
# closer look at lowest diff
lotdf.head(10)

,LotFrontage,LotArea,ifsqare,diff
934,313.0,27650,166.282891,-146.717109
1127,182.0,14572,120.714539,-61.285461
1298,313.0,63887,252.758778,-60.241222
231,174.0,15138,123.036580,-50.963420
1211,152.0,12134,110.154437,-41.845563
909,149.0,12589,112.200713,-36.799287
1182,160.0,15623,124.992000,-35.008000
276,129.0,9196,95.895777,-33.104223
966,130.0,9600,97.979590,-32.020410
429,130.0,11457,107.037377,-22.962623


In [37]:
###################################################################################################################
### NOTES:
# 
# observation:
#       - variaty of differences too big to make a conclusion via area shape
# decision:
#       - confirm via ols before filling na in wdf with median
#
###################################################################################################################

In [38]:
# OLS LotArea, LotFrontage

    # add constant
lotdf = sm.add_constant(lotdf)
    
    # ols
results = sm.OLS(lotdf["LotFrontage"], 
                  lotdf[["LotArea", "const"]]).fit()
    # summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            LotFrontage   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     233.5
Date:                Tue, 20 Oct 2020   Prob (F-statistic):           6.43e-48
Time:                        15:06:31   Log-Likelihood:                -4945.6
No. Observations:                1094   AIC:                             9895.
Df Residuals:                    1092   BIC:                             9905.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
LotArea        0.0013    8.2e-05     15.281      0.000       0.001       0.001
const         58.0678      1.069     54.323      0.000      55.970      60.165
==============================================================================
Omnibus:                      436.477   Durbin-Watson:                   0.642
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12811.700
Skew:                           1.214   Prob(JB):                         0.00
Kurtosis:                      19.588   Cond. No.                     2.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
###################################################################################################################
### SUMMARY Frontage/Area:
#
# LotFronatge vals:
#                  - std: 24.508858570320566
#                  - mean: 70.75959780621572
#                  - median: 70.0
#                  - min: 21.0
#                  - max: 313.0
#
# OLS Frontage/Area vals:
#                  - coef: 0.0013
#                  - Adj. R-squared: 0.175
#                  - t: 15.281
#
###################################################################################################################

In [40]:
###################################################################################################################
### NOTES:
#
# conclusion: OLS confirmed that compared area vals not sufficient to calculate missing frontage vals
#
# decision: missing val will be filled with mean
#
###################################################################################################################

# store mean 
fmean = wdf['LotFrontage'].mean()

# fillna and update
wdf['LotFrontage'] = wdf['LotFrontage'].fillna(fmean)

###################################################################################################################
###################################################################################################################

In [41]:
###################################################################################################################
# confirm: no more remaining na vals in wdf

confirm = pd.DataFrame(wdf.isna().sum())
len(confirm.loc[confirm[0]>1])==0

###################################################################################################################

True

# Dtypes

In [42]:
    # make a temp copy of wdf to be save
temp = wdf.copy()

    # review dtypes and store them to df
temptypes = pd.DataFrame(temp.dtypes)
temptypes = temptypes.loc[temptypes[0]==object]
    # review:
temptypes
    

,0
MSZoning,object
Street,object
LotShape,object
LandContour,object
Utilities,object
LotConfig,object
LandSlope,object
Neighborhood,object
Condition1,object
Condition2,object


In [45]:
wdf = pd.get_dummies(wdf)
wdf.head()

,const,id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1404,1.0,1405,50,60.000000,10410,3,4,1915,1950,0.0,...,0,0,0,1,0,0,0,1,0,0
996,1.0,997,20,70.759598,10659,5,6,1961,1961,0.0,...,0,0,0,0,0,0,0,0,1,0
411,1.0,412,190,100.000000,34650,5,5,1955,1955,0.0,...,0,0,0,1,0,0,0,0,1,0
1040,1.0,1041,20,88.000000,13125,5,4,1957,2000,67.0,...,0,0,0,1,0,0,0,0,1,0
370,1.0,371,60,70.759598,8121,6,5,2000,2000,0.0,...,0,0,0,1,0,0,0,0,1,0


In [61]:
# make a new wdf copy = c(leaned)wdf
cwdf = wdf.copy()
cwdf.columns.to_list()

['const',
 'id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'month',
 'year',
 'price',
 'date',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Grvl',
 'Street_Pave',
 'LotShape_IR1',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_Bnk',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'Utilities_AllPub',
 'Utilities_NoSeWa',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Gtl',
 'Land

In [77]:
# combining Central Air Cols
cwdf['CentralAir'] = cwdf['CentralAir_Y']
cwdf.drop(columns={'CentralAir_Y','CentralAir_N'})

,const,id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,CentralAir
1404,1.0,1405,50,60.000000,10410,3,4,1915,1950,0.0,...,0,0,1,0,0,0,1,0,0,1
996,1.0,997,20,70.759598,10659,5,6,1961,1961,0.0,...,0,0,0,0,0,0,0,1,0,1
411,1.0,412,190,100.000000,34650,5,5,1955,1955,0.0,...,0,0,1,0,0,0,0,1,0,0
1040,1.0,1041,20,88.000000,13125,5,4,1957,2000,67.0,...,0,0,1,0,0,0,0,1,0,1
370,1.0,371,60,70.759598,8121,6,5,2000,2000,0.0,...,0,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,1.0,1145,190,60.000000,12180,4,4,1941,1950,0.0,...,0,0,1,0,0,0,0,1,0,0
1421,1.0,1422,120,53.000000,4043,6,5,1977,1977,0.0,...,0,0,1,0,0,0,0,1,0,1
419,1.0,420,20,65.000000,8450,5,6,1968,1968,0.0,...,0,0,1,0,0,0,0,1,0,1
1160,1.0,1161,160,24.000000,2280,6,5,1978,1978,0.0,...,0,0,1,0,0,0,0,1,0,1


In [78]:
cwdf.columns.to_list()

['const',
 'id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'month',
 'year',
 'price',
 'date',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Grvl',
 'Street_Pave',
 'LotShape_IR1',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_Bnk',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_Lvl',
 'Utilities_AllPub',
 'Utilities_NoSeWa',
 'LotConfig_Corner',
 'LotConfig_CulDSac',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'LotConfig_Inside',
 'LandSlope_Gtl',
 'Land

In [93]:
#cwdf["Utilities"] = cwdf['Utilities_AllPub']
#cwdf['Utilities_NoSeWa'] != cwdf["Utilities"]
#cwdf.drop(columns={"Utilities_NoSeWa"})

In [99]:
cwdf['Street_1Pave_0Grvl'] = cwdf['Street_Pave']

In [100]:
cwdf.drop(columns= {'Street_Grvl','Street_Pave'})

,const,id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,CentralAir,Utilities,Street_1Pave_0Grvl
1404,1.0,1405,50,60.000000,10410,3,4,1915,1950,0.0,...,1,0,0,0,1,0,0,1,1,1
996,1.0,997,20,70.759598,10659,5,6,1961,1961,0.0,...,0,0,0,0,0,1,0,1,1,1
411,1.0,412,190,100.000000,34650,5,5,1955,1955,0.0,...,1,0,0,0,0,1,0,0,1,1
1040,1.0,1041,20,88.000000,13125,5,4,1957,2000,67.0,...,1,0,0,0,0,1,0,1,1,1
370,1.0,371,60,70.759598,8121,6,5,2000,2000,0.0,...,1,0,0,0,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,1.0,1145,190,60.000000,12180,4,4,1941,1950,0.0,...,1,0,0,0,0,1,0,0,1,1
1421,1.0,1422,120,53.000000,4043,6,5,1977,1977,0.0,...,1,0,0,0,0,1,0,1,1,1
419,1.0,420,20,65.000000,8450,5,6,1968,1968,0.0,...,1,0,0,0,0,1,0,1,1,1
1160,1.0,1161,160,24.000000,2280,6,5,1978,1978,0.0,...,1,0,0,0,0,1,0,1,1,1


In [143]:
####################################################################################################################
# nobinary object cols:
temp = cwdf.copy()
cols = [x for x in temp.columns]
#c = cols[5]
#len(temp[c].unique())


9

In [148]:
nobincols = []
valcount = []
for col in cols:
    if len(temp[col].unique())>2:
        nobincols.append(col)
        valcount.append(len(temp[col].unique()))
        
d = {"NonbinCols": nobincols, "ValCount": valcount}

In [156]:
nonbindf = pd.DataFrame(d)
nonbindf.head()

,NonbinCols,ValCount
0,id,1338
1,MSSubClass,15
2,LotFrontage,108
3,LotArea,1000
4,OverallQual,9


In [183]:
def replacer(col):
    coldf = pd.DataFrame(temp[x].value_counts()).reset_index().rename(columns={'index':'value',x:'count'}).sort_values('count', ascending=False)
    coldf['newval'] = coldf.index+1
    newvals = coldf['newval'].to_list()
    oldvals = coldf['value'].to_list()
    dictreplace = dict(zip(oldvals,newvals))
    return dictreplace

In [187]:
# do through val replacements manually and print each vals to check

# copy cwdf to ccdf:
ccdf = cwdf.copy()

In [188]:
i = 0
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

1 id {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 100, 18

In [189]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

2 MSSubClass {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917:

In [190]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

3 LotFrontage {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [191]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

4 LotArea {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 10

In [192]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

5 OverallQual {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [193]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

6 OverallCond {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [194]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

7 YearBuilt {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 

In [195]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

8 YearRemodAdd {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 191

In [256]:
# val error for i=9

col = 'HouseStyle'
ndf = pd.DataFrame(ccdf[col].value_counts()).reset_index().reset_index().rename(columns={'index':'col'})
ndf['rank'] = ndf['level_0']+1
oldv = ndf['col'].to_list()
nval = ndf['rank'].to_list()
d = dict(zip(oldv,nval))
ccdf[col] = ccdf[col].replace(d)

HouseStyle_1.5Fin',
 'HouseStyle_1.5Unf',
 'HouseStyle_1Story',
 'HouseStyle_2.5Fin',
 'HouseStyle_2.5Unf',
 'HouseStyle_2Story',
 'HouseStyle_SFoyer',
 'HouseStyle_SLvl',

In [258]:
# continue with i = 10

i = 10
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

11 BsmtFinSF2 {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [259]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

12 BsmtUnfSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917:

In [260]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

13 TotalBsmtSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 191

In [261]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

14 1stFlrSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 

In [262]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

15 2ndFlrSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 

In [263]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

16 LowQualFinSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [264]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

17 GrLivArea {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917:

In [265]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

18 BsmtFullBath {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [266]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

19 BsmtHalfBath {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [267]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

20 FullBath {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 

In [268]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

21 HalfBath {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917: 

In [269]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

22 BedroomAbvGr {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [270]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

23 KitchenAbvGr {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [271]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

24 TotRmsAbvGrd {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 19

In [272]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

25 Fireplaces {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [273]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

26 GarageYrBlt {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 191

In [274]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

27 GarageCars {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [275]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

28 GarageArea {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [276]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

29 WoodDeckSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917

In [277]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

30 OpenPorchSF {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 191

In [278]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

31 EnclosedPorch {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1

In [279]:
c = nobincols[i] 
d = replacerlist[i]
ccdf[c] = ccdf[c].replace(d)
i +=1
print(i,c,d)

32 3SsnPorch {2005: 1, 2006: 2, 2004: 3, 2007: 4, 2003: 5, 1976: 6, 1977: 7, 1920: 8, 1999: 9, 1998: 10, 2000: 11, 1954: 14, 2008: 15, 1958: 13, 1965: 12, 1959: 16, 1968: 17, 1970: 18, 1972: 19, 2002: 20, 2001: 21, 1994: 25, 1995: 26, 1962: 27, 2009: 24, 1971: 23, 1966: 22, 1957: 29, 1993: 28, 1960: 30, 1963: 31, 1925: 32, 1967: 33, 1950: 34, 1996: 35, 1964: 36, 1978: 40, 1969: 39, 1940: 38, 1997: 37, 1961: 41, 1948: 43, 1941: 44, 1992: 42, 1955: 45, 1953: 46, 1956: 47, 1910: 48, 1988: 49, 1990: 50, 1973: 51, 1980: 52, 1984: 54, 1915: 55, 1974: 53, 1926: 56, 1936: 57, 1900: 58, 1939: 59, 1949: 60, 1928: 61, 1930: 62, 1916: 63, 1918: 64, 1975: 65, 1979: 70, 1982: 69, 1923: 68, 1921: 67, 1951: 66, 1922: 71, 1924: 72, 1986: 73, 1937: 74, 1981: 75, 1931: 76, 1985: 87, 1952: 86, 1946: 85, 1945: 84, 1938: 83, 1935: 82, 1991: 81, 1983: 80, 1932: 79, 1880: 78, 1929: 77, 1989: 88, 1934: 89, 1914: 90, 1947: 91, 1927: 92, 1892: 99, 1919: 98, 1890: 97, 1908: 96, 1942: 95, 1885: 94, 1987: 93, 1917:

In [280]:
# copy ccdf to findf

cleandf = ccdf.copy()

####################################################################################################################

In [281]:
####################################################################################################################
####################################################################################################################
### EXPORT:
#
# cleandf to csv

cleandf.to_csv('cleandata.csv')

####################################################################################################################
####################################################################################################################